In [1]:
import cv2
import numpy as np
import time
import imutils

In [2]:
def to_channels(img):
    img = cv2.UMat(img)
    img = cv2.GaussianBlur(img, (3,3), sigmaX=0, sigmaY=0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
    gradX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1 )
    gradY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1 )
    mag, ori = cv2.cartToPolar(gradX, gradY, angleInDegrees=True)

    ori_ranges = [(0,60), (60,120), (120,180), (180,240), (240,300), (300,360)]
    oris=[cv2.inRange(ori,l,r) for (l,r) in ori_ranges]
    mags= [cv2.copyTo(mag, ori_x) for ori_x in oris]
    channels=[gray,mag]+mags
    
    ###save
    cv2.imwrite("gray.png", gray)
    cv2.imwrite("mag.png", mag)
    cv2.imwrite("ori.png", ori)
    cv2.imwrite("gradX.png", gradX)
    cv2.imwrite("gradY.png", gradY)
    for i in range(len(oris)):
        cv2.imwrite("ori"+str(60*i + 30)+".png", oris[i])
        cv2.imwrite("mag"+str(60*i + 30)+".png", mags[i])
    ###save
    
    return channels

def aggregate_sum_per_channel(ch):
    row_aggr=np.add(ch[::2],ch[1::2])
    full_aggr=np.add(row_aggr[:,::2],row_aggr[:,1::2])
    return full_aggr

def aggregate_sum(umat_channels):
    return [aggregate_sum_per_channel(cv2.UMat.get(ch)) for ch in umat_channels]

def get_aggregates(img):
    channels = to_channels(img)
    aggregates = aggregate_sum(channels)
    aggregates = np.array([cv2.GaussianBlur(aggr, (3,3), sigmaX=0, sigmaY=0) for aggr in aggregates])

    for i in range(len(aggregates)):
        cv2.imwrite("aggr_ch"+str(i)+".png", aggregates[i])
        
    return aggregates

In [3]:
img = cv2.imread('real_pallet.jpg')
aggregates = get_aggregates(img)
features = aggregates.flatten()


# Sliding window

In [4]:
def pyramid(image, scale=1.5, minSize=(30, 30)):
    yield image
    while True:
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        yield image
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [ ]:
image = aggregates[1]/255.0 #for displaying purposes /255.0
(winW, winH) = (120,24)

for resized in pyramid(image, scale=1.5):
    for (x, y, window) in sliding_window(resized, stepSize=32, windowSize=(winW, winH)):
        # if the window does not meet our desired window size, ignore it
        if window.shape[0] != winH or window.shape[1] != winW:
            continue
        #classify()
        clone = resized.copy()
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
        cv2.imshow("Window", clone)
        cv2.waitKey(1)
        #time.sleep(0.025)

In [ ]:
Oni mają 12 skal co 1.15, trzeba tak dobrać dataset, zeby też tak było - 0d 0.2 do 1.0

1. Jak dobrać kąty, czy po 2 syetryczne czy po 1?
2. Czy skalować obraz wejsciowy czy kanały?
3. I jak sklaować ??
4. czy cupy przyspieszy agregację? #cupy.array/cupy.asnumpy
5. czy użyć buildPyramid z opencv?

Parametry:
1. kernel rozmazania (w publikacji [1,2,1]/4)
2. wielkość agregatów - 2x2, 4x4
3. wielkość ramki 24 x 120
4. wielkość palety 20 x 100